In [2]:
# CELL 1: Install dependencies
!pip install timm albumentations efficientnet_pytorch torch torchvision tqdm scikit-learn pandas numpy pillow


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# CELL 2: Imports & Configuration
import os, time, json, warnings, glob
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingWarmRestarts
from sklearn.metrics import roc_auc_score
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import timm

# ── Device ──
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU : {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")

# ── Constants ──
IMG_SIZE     = 224
BATCH_SIZE   = 10        # safe for RTX 3070 Ti 8 GB
NUM_WORKERS  = 0         # Windows → must be 0
SEED         = 42
LABEL_SMOOTH = 0.1

LABELS = [
    "Atelectasis","Consolidation","Infiltration","Pneumothorax",
    "Edema","Emphysema","Fibrosis","Effusion","Pneumonia",
    "PleuralThickening","Cardiomegaly","Nodule","Mass","Hernia"
]
NUM_CLASSES = len(LABELS)

# ── YOUR Paths ──
BASE_DIR   = r"D:\Major Project"
DATA_DIR   = r"D:\Major Project\chest_xray_data"   # contains images_001 … images_012
CSV_FILE   = os.path.join(DATA_DIR, "Data_Entry_2017.csv")
TRAIN_LIST = os.path.join(DATA_DIR, "train_val_list.txt")
TEST_LIST  = os.path.join(DATA_DIR, "test_list.txt")

# ── Saved model paths (already trained) ──
PATH_B0   = os.path.join(BASE_DIR, "best_balanced_effnet.pth")
PATH_B3   = os.path.join(BASE_DIR, "best_efficientnet_b3.pth")
PATH_VIT  = os.path.join(BASE_DIR, "best_vit_base.pth")
PATH_DINO = os.path.join(BASE_DIR, "best_ultimate_dinov2.pth")  # will be saved here after Phase 7

torch.manual_seed(SEED)
np.random.seed(SEED)
print(f"\nLabels ({NUM_CLASSES}): {LABELS}")
print(f"\nData dir   : {DATA_DIR}")
print(f"CSV        : {CSV_FILE}")
print(f"B0  weights: {PATH_B0}")
print(f"B3  weights: {PATH_B3}")
print(f"ViT weights: {PATH_VIT}")


Using device: cuda
GPU : NVIDIA GeForce RTX 3070 Ti Laptop GPU
VRAM: 8.6 GB

Labels (14): ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'PleuralThickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia']

Data dir   : D:\Major Project\chest_xray_data
CSV        : D:\Major Project\chest_xray_data\Data_Entry_2017.csv
B0  weights: D:\Major Project\best_balanced_effnet.pth
B3  weights: D:\Major Project\best_efficientnet_b3.pth
ViT weights: D:\Major Project\best_vit_base.pth


In [4]:
# CELL 3: Build filename → full_path lookup (Windows-safe, multi-method)

import os, glob
from pathlib import Path

def build_image_map(data_dir):
    img_map = {}
    data_path = Path(data_dir)
    
    print(f"Scanning: {data_path}")
    print(f"Directory exists: {data_path.exists()}")
    
    if not data_path.exists():
        print("❌ ERROR: DATA_DIR does not exist! Check the path.")
        return img_map
    
    # List what's directly inside
    top_level = list(data_path.iterdir())
    print(f"Top-level items found: {len(top_level)}")
    for item in top_level[:5]:
        print(f"  → {item.name}  (dir={item.is_dir()})")
    
    # Method 1: Walk all subdirectories recursively
    for root, dirs, files in os.walk(data_path):
        for fname in files:
            if fname.lower().endswith(".png"):
                img_map[fname] = os.path.join(root, fname)
    
    print(f"\nTotal .png images found: {len(img_map)}")
    
    if len(img_map) == 0:
        # Debug: show what files ARE present
        print("\n⚠️  No PNGs found. Checking actual contents:")
        for root, dirs, files in os.walk(data_path):
            if files:
                print(f"  Folder: {root}")
                print(f"  Sample files: {files[:3]}")
                break
    else:
        # Show sample entries
        sample = list(img_map.items())[:3]
        print("Sample entries:")
        for k, v in sample:
            print(f"  {k} → {v}")
    
    return img_map

IMAGE_MAP = build_image_map(DATA_DIR)



Scanning: D:\Major Project\chest_xray_data
Directory exists: True
Top-level items found: 20
  → ARXIV_V5_CHESTXRAY.pdf  (dir=False)
  → BBox_List_2017.csv  (dir=False)
  → Data_Entry_2017.csv  (dir=False)
  → FAQ_CHESTXRAY.pdf  (dir=False)
  → images_001  (dir=True)

Total .png images found: 112120
Sample entries:
  00000001_000.png → D:\Major Project\chest_xray_data\images_001\images\00000001_000.png
  00000001_001.png → D:\Major Project\chest_xray_data\images_001\images\00000001_001.png
  00000001_002.png → D:\Major Project\chest_xray_data\images_001\images\00000001_002.png


In [5]:
# CELL 4: Split Data_Entry_2017.csv using official train_val / test lists

def load_dataframes():
    df = pd.read_csv(CSV_FILE)
    df.rename(columns={"Image Index": "Image_Index",
                        "Finding Labels": "Finding Labels",
                        "Patient Age": "Patient_Age",
                        "Patient Gender": "Patient_Gender",
                        "View Position": "View_Position",
                        "Follow-up #": "Follow-up #"}, inplace=True)

    with open(TRAIN_LIST) as f:
        train_val_files = set(line.strip() for line in f)
    with open(TEST_LIST) as f:
        test_files = set(line.strip() for line in f)

    df_trainval = df[df["Image_Index"].isin(train_val_files)].reset_index(drop=True)
    df_test     = df[df["Image_Index"].isin(test_files)].reset_index(drop=True)

    # 85/15 split of train_val → train / val
    from sklearn.model_selection import train_test_split
    df_train, df_val = train_test_split(
        df_trainval, test_size=0.15, random_state=SEED, shuffle=True
    )
    df_train = df_train.reset_index(drop=True)
    df_val   = df_val.reset_index(drop=True)

    print(f"Train : {len(df_train):>6} images")
    print(f"Val   : {len(df_val):>6} images")
    print(f"Test  : {len(df_test):>6} images")
    return df_train, df_val, df_test

DF_TRAIN, DF_VAL, DF_TEST = load_dataframes()


Train :  73545 images
Val   :  12979 images
Test  :  25596 images


In [6]:
# CELL 5: ChestXrayDataset, transforms, weighted sampler

def build_transforms(train=True):
    norm = A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225))
    if train:
        return A.Compose([
            A.Resize(IMG_SIZE, IMG_SIZE),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(0.15, 0.15, p=0.5),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05,
                               rotate_limit=5, p=0.4),
            A.GaussNoise(var_limit=(5,20), p=0.2),
            norm, ToTensorV2(),
        ])
    return A.Compose([A.Resize(IMG_SIZE, IMG_SIZE), norm, ToTensorV2()])


class ChestXrayDataset(Dataset):
    def __init__(self, df, image_map, transform=None):
        # Keep only rows whose image file was actually found on disk
        self.df        = df[df["Image_Index"].isin(image_map)].reset_index(drop=True)
        self.image_map = image_map
        self.transform = transform
        skipped = len(df) - len(self.df)
        if skipped:
            print(f"  ⚠️  Skipped {skipped} rows (image not found on disk)")
        print(f"  Dataset ready: {len(self.df)} samples")

    def _encode_metadata(self, row):
        try:
            age = float(str(row.get("Patient_Age", 50)).replace("Y","").strip())
        except:
            age = 50.0
        age    = np.clip(age, 0, 95) / 95.0
        gender = {"M":0.0,"F":1.0}.get(str(row.get("Patient_Gender","U")).upper(), 0.5)
        view   = {"PA":0.0,"AP":1.0}.get(str(row.get("View_Position","U")).upper(), 0.5)
        follow = np.clip(float(row.get("Follow-up #", 0)) / 10.0, 0, 1)
        return np.array([age, gender, view, follow], dtype=np.float32)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row      = self.df.iloc[idx]
        img_path = self.image_map[row["Image_Index"]]
        img      = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            img = self.transform(image=img)["image"]

        label_vec = np.zeros(NUM_CLASSES, dtype=np.float32)
        for lbl in str(row.get("Finding Labels","No Finding")).split("|"):
            lbl = lbl.strip()
            if lbl in LABELS:
                label_vec[LABELS.index(lbl)] = 1.0

        metadata = self._encode_metadata(row)
        return img, torch.FloatTensor(label_vec), torch.FloatTensor(metadata)


def build_weighted_sampler(df):
    disease_w = {
        "Hernia":50.0,"Pneumonia":15.0,"Fibrosis":8.0,
        "Edema":7.0,"Emphysema":6.0,"Cardiomegaly":5.0,
        "PleuralThickening":4.0,"Nodule":3.0,"Mass":3.0,
        "Consolidation":2.5,"Pneumothorax":2.0,
        "Effusion":1.5,"Atelectasis":1.2,"Infiltration":1.0,
    }
    sample_weights = []
    for _, row in df.iterrows():
        raw      = str(row.get("Finding Labels","No Finding"))
        diseases = [l.strip() for l in raw.split("|") if l.strip() in LABELS]
        w = max((disease_w.get(d,1.0) for d in diseases), default=0.2)
        sample_weights.append(w)
    return WeightedRandomSampler(sample_weights,
                                  num_samples=len(sample_weights),
                                  replacement=True)


def build_dataloaders():
    train_ds = ChestXrayDataset(DF_TRAIN, IMAGE_MAP, build_transforms(True))
    val_ds   = ChestXrayDataset(DF_VAL,   IMAGE_MAP, build_transforms(False))
    test_ds  = ChestXrayDataset(DF_TEST,  IMAGE_MAP, build_transforms(False))

    sampler  = build_weighted_sampler(DF_TRAIN)
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler,
                          num_workers=NUM_WORKERS, pin_memory=True)
    val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
    test_dl  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
    print(f"\nBatches → Train:{len(train_dl)}  Val:{len(val_dl)}  Test:{len(test_dl)}")
    return train_dl, val_dl, test_dl

TRAIN_DL, VAL_DL, TEST_DL = build_dataloaders()


  Dataset ready: 73545 samples
  Dataset ready: 12979 samples
  Dataset ready: 25596 samples

Batches → Train:7355  Val:1298  Test:2560


In [7]:
# CELL 6: Focal Loss with Label Smoothing + Class Weight Calculator

def compute_class_weights(df):
    counts = np.zeros(NUM_CLASSES)
    for _, row in df.iterrows():
        for lbl in str(row.get("Finding Labels","")).split("|"):
            lbl = lbl.strip()
            if lbl in LABELS:
                counts[LABELS.index(lbl)] += 1
    freq    = np.clip(counts / len(df), 1e-4, 1.0)
    weights = 1.0 / freq
    weights = weights / weights.mean()
    print("Class weights:")
    for l, w in zip(LABELS, weights):
        print(f"  {l:<20} {w:.2f}")
    return torch.FloatTensor(weights)


class FocalLossWithSmoothing(nn.Module):
    def __init__(self, class_weights, gamma=2.0, smoothing=0.1):
        super().__init__()
        self.class_weights = class_weights
        self.gamma         = gamma
        self.smoothing     = smoothing

    def forward(self, logits, targets):
        targets_smooth = targets * (1 - self.smoothing) + 0.5 * self.smoothing
        bce   = F.binary_cross_entropy_with_logits(logits, targets_smooth, reduction="none")
        probs = torch.sigmoid(logits)
        pt    = torch.where(targets > 0.5, probs, 1 - probs)
        gammas = torch.full((NUM_CLASSES,), self.gamma, device=logits.device)
        gammas[8]  = 3.0   # Pneumonia
        gammas[11] = 2.5   # Nodule
        focal_w = (1 - pt) ** gammas
        return (focal_w * bce * self.class_weights.to(logits.device)).mean()


CLASS_WEIGHTS = compute_class_weights(DF_TRAIN)


Class weights:
  Atelectasis          0.01
  Consolidation        0.04
  Infiltration         0.01
  Pneumothorax         0.04
  Edema                0.08
  Emphysema            0.08
  Fibrosis             0.09
  Effusion             0.01
  Pneumonia            0.13
  PleuralThickening    12.62
  Cardiomegaly         0.07
  Nodule               0.02
  Mass                 0.03
  Hernia               0.78


In [8]:
# CELL 7: All model architectures

# ── EfficientNet-B0 ──
class EfficientNetB0(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, dropout=0.3):
        super().__init__()
        self.backbone = timm.create_model("efficientnet_b0", pretrained=False, num_classes=0)
        feat_dim = self.backbone.num_features
        self.head = nn.Sequential(
            nn.Linear(feat_dim, 512), nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes),
        )
    def forward(self, x, metadata=None):
        return self.head(self.backbone(x))


# ── EfficientNet-B3 ──
class EfficientNetB3(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, dropout=0.4):
        super().__init__()
        self.backbone = timm.create_model("efficientnet_b3", pretrained=False, num_classes=0)
        feat_dim = self.backbone.num_features
        self.head = nn.Sequential(
            nn.Linear(feat_dim, 512), nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes),
        )
    def forward(self, x, metadata=None):
        return self.head(self.backbone(x))


# ──────────────────────────────────────────────────────────────────
# ViT-Base/16  — matches YOUR saved best_vit_base.pth exactly
#
# Saved keys reveal original structure:
#   self.vit        → timm ViT backbone  (vit.blocks, vit.norm)
#   self.classifier → Sequential head   (classifier.0, .2, .5)
#
# Head structure from keys:
#   classifier.0  → Linear (LayerNorm not saved separately → it's Linear)
#   classifier.2  → Linear
#   classifier.5  → Linear   (3 linear layers with indices 0, 2, 5)
#   → Sequential: Linear, act, Linear, Dropout, act, Linear  etc.
# ──────────────────────────────────────────────────────────────────

class ViTBase(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, dropout=0.3):
        super().__init__()
        self.vit = timm.create_model(
            "vit_base_patch16_224", pretrained=False, num_classes=0
        )
        feat_dim = self.vit.num_features  # 768

        # Exactly 3 Linear layers at indices 0, 2, 5
        # Sequential: Linear(768,512), GELU, Linear(512,256), Dropout, GELU, Linear(256,14)
        self.classifier = nn.Sequential(
            nn.Linear(feat_dim, 512),   # index 0
            nn.GELU(),                  # index 1
            nn.Linear(512, 256),        # index 2
            nn.Dropout(dropout),        # index 3
            nn.GELU(),                  # index 4
            nn.Linear(256, num_classes) # index 5
        )

    def forward(self, x, metadata=None):
        return self.classifier(self.vit(x))


# ── DINOv2 Multi-Modal (Phase 7) ──
class AnatomicalAttention(nn.Module):
    def __init__(self, feat_dim=768):
        super().__init__()
        self.gate = nn.Sequential(
            nn.Linear(feat_dim, feat_dim // 4), nn.ReLU(),
            nn.Linear(feat_dim // 4, feat_dim), nn.Sigmoid(),
        )
    def forward(self, spatial_feats):
        gap  = spatial_feats.mean(dim=[2, 3])
        gate = self.gate(gap)
        return gap * gate


class DINOv2MultiModal(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, freeze_blocks=8, dropout=0.3):
        super().__init__()
        self.backbone = torch.hub.load(
            "facebookresearch/dinov2", "dinov2_vitb14", pretrained=True
        )
        for p in self.backbone.parameters():
            p.requires_grad = False
        for i in range(freeze_blocks, 12):
            for p in self.backbone.blocks[i].parameters():
                p.requires_grad = True
        for p in self.backbone.norm.parameters():
            p.requires_grad = True

        feat_dim = self.backbone.embed_dim  # 768
        self.attn = AnatomicalAttention(feat_dim)

        self.meta_encoder = nn.Sequential(
            nn.Linear(4, 64),   nn.ReLU(),
            nn.Linear(64, 128), nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU(),
        )
        fuse_dim = feat_dim + 256  # 1024
        self.fusion_head = nn.Sequential(
            nn.LayerNorm(fuse_dim),
            nn.Linear(fuse_dim, 512), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(512, 256),      nn.GELU(), nn.Dropout(dropout * 0.67),
            nn.Linear(256, num_classes),
        )

    def forward(self, images, metadata):
        B = images.shape[0]
        out          = self.backbone.forward_features(images)
        patch_tokens = out["x_norm_patchtokens"]
        H = W        = int(patch_tokens.shape[1] ** 0.5)
        spatial      = patch_tokens.permute(0,2,1).reshape(B, -1, H, W)
        cls_token    = out["x_norm_clstoken"]
        attended     = self.attn(spatial)
        img_feat     = attended + cls_token
        meta_feat    = self.meta_encoder(metadata)
        fused        = torch.cat([img_feat, meta_feat], dim=1)
        return self.fusion_head(fused)


def count_params(model, name):
    total    = sum(p.numel() for p in model.parameters()) / 1e6
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6
    print(f"{name:<22} {total:.1f}M total | {trainable:.1f}M trainable")


In [9]:
# CELL 7: Model definitions matching YOUR saved .pth files exactly
# B0 & B3 → efficientnet_pytorch  (net._blocks keys)
# ViT      → timm                  (backbone.blocks keys)

import torch.nn as nn
import timm
from efficientnet_pytorch import EfficientNet

# ──────────────────────────────────────────────
# EfficientNet-B0  (efficientnet_pytorch style)
# Keys: net._conv_stem, net._blocks, net._fc
# ──────────────────────────────────────────────
class EfficientNetB0(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, dropout=0.3):
        super().__init__()
        self.net = EfficientNet.from_pretrained("efficientnet-b0")
        in_features = self.net._fc.in_features
        self.net._fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_features, num_classes),
        )

    def forward(self, x, metadata=None):
        return self.net(x)


# ──────────────────────────────────────────────
# EfficientNet-B3  (efficientnet_pytorch style)
# Keys: net._conv_stem, net._blocks, net._fc
# ──────────────────────────────────────────────
class EfficientNetB3(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, dropout=0.4):
        super().__init__()
        self.net = EfficientNet.from_pretrained("efficientnet-b3")
        in_features = self.net._fc.in_features
        self.net._fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x, metadata=None):
        return self.net(x)

class ViTBase(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, dropout=0.3):
        super().__init__()
        self.vit = timm.create_model(
            "vit_base_patch16_224", pretrained=False, num_classes=0
        )
        feat_dim = self.vit.num_features  # 768

        # Exact structure decoded from checkpoint shapes:
        # classifier.0 → LayerNorm(768)       weight shape (768,)
        # classifier.1 → Dropout              no weights, index 1
        # classifier.2 → Linear(768 → 512)    weight shape (512, 768)
        # classifier.3 → GELU                 no weights, index 3
        # classifier.4 → Dropout              no weights, index 4
        # classifier.5 → Linear(512 → 14)     weight shape (14, 512)
        self.classifier = nn.Sequential(
            nn.LayerNorm(feat_dim),     # 0  ← weight shape (768,) ✅
            nn.Dropout(dropout),        # 1
            nn.Linear(feat_dim, 512),   # 2  ← weight shape (512, 768) ✅
            nn.GELU(),                  # 3
            nn.Dropout(dropout * 0.67), # 4
            nn.Linear(512, num_classes) # 5  ← weight shape (14, 512) ✅
        )

    def forward(self, x, metadata=None):
        return self.classifier(self.vit(x))




# ──────────────────────────────────────────────
# DINOv2 Multi-Modal  (Phase 7 — trained fresh)
# ──────────────────────────────────────────────
class AnatomicalAttention(nn.Module):
    def __init__(self, feat_dim=768):
        super().__init__()
        self.gate = nn.Sequential(
            nn.Linear(feat_dim, feat_dim // 4), nn.ReLU(),
            nn.Linear(feat_dim // 4, feat_dim), nn.Sigmoid(),
        )
    def forward(self, spatial_feats):
        gap  = spatial_feats.mean(dim=[2, 3])
        return gap * self.gate(gap)


class DINOv2MultiModal(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, freeze_blocks=8, dropout=0.3):
        super().__init__()
        self.backbone = torch.hub.load(
            "facebookresearch/dinov2", "dinov2_vitb14", pretrained=True
        )
        for p in self.backbone.parameters():
            p.requires_grad = False
        for i in range(freeze_blocks, 12):
            for p in self.backbone.blocks[i].parameters():
                p.requires_grad = True
        for p in self.backbone.norm.parameters():
            p.requires_grad = True

        feat_dim = self.backbone.embed_dim  # 768
        self.attn = AnatomicalAttention(feat_dim)
        self.meta_encoder = nn.Sequential(
            nn.Linear(4, 64),    nn.ReLU(),
            nn.Linear(64, 128),  nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU(),
        )
        self.fusion_head = nn.Sequential(
            nn.LayerNorm(feat_dim + 256),
            nn.Linear(feat_dim + 256, 512), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(512, 256),            nn.GELU(), nn.Dropout(dropout * 0.67),
            nn.Linear(256, num_classes),
        )

    def forward(self, images, metadata):
        B   = images.shape[0]
        out = self.backbone.forward_features(images)
        patch_tokens = out["x_norm_patchtokens"]
        H = W        = int(patch_tokens.shape[1] ** 0.5)
        spatial      = patch_tokens.permute(0, 2, 1).reshape(B, -1, H, W)
        cls_token    = out["x_norm_clstoken"]
        img_feat     = self.attn(spatial) + cls_token
        meta_feat    = self.meta_encoder(metadata)
        return self.fusion_head(torch.cat([img_feat, meta_feat], dim=1))


def count_params(model, name):
    total     = sum(p.numel() for p in model.parameters()) / 1e6
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6
    print(f"{name:<25} {total:.1f}M total | {trainable:.1f}M trainable")


In [32]:
# KEY INSPECTOR — run once to see exact ViT saved structure
ckpt = torch.load(PATH_VIT, map_location="cpu")
if isinstance(ckpt, dict) and "model_state_dict" in ckpt:
    ckpt = ckpt["model_state_dict"]
elif isinstance(ckpt, dict) and "state_dict" in ckpt:
    ckpt = ckpt["state_dict"]

print("=== ALL ViT checkpoint keys ===")
for k, v in ckpt.items():
    print(f"  {k:<45} shape: {tuple(v.shape)}")


=== ALL ViT checkpoint keys ===
  vit.cls_token                                 shape: (1, 1, 768)
  vit.pos_embed                                 shape: (1, 197, 768)
  vit.patch_embed.proj.weight                   shape: (768, 3, 16, 16)
  vit.patch_embed.proj.bias                     shape: (768,)
  vit.blocks.0.norm1.weight                     shape: (768,)
  vit.blocks.0.norm1.bias                       shape: (768,)
  vit.blocks.0.attn.qkv.weight                  shape: (2304, 768)
  vit.blocks.0.attn.qkv.bias                    shape: (2304,)
  vit.blocks.0.attn.proj.weight                 shape: (768, 768)
  vit.blocks.0.attn.proj.bias                   shape: (768,)
  vit.blocks.0.norm2.weight                     shape: (768,)
  vit.blocks.0.norm2.bias                       shape: (768,)
  vit.blocks.0.mlp.fc1.weight                   shape: (3072, 768)
  vit.blocks.0.mlp.fc1.bias                     shape: (3072,)
  vit.blocks.0.mlp.fc2.weight                   shape: (768, 3

In [13]:
# CELL 8: Load pre-trained B0, B3, ViT — with auto key-inspection before loading

def inspect_and_load(model, path, name):
    """
    Inspects the first 3 keys of the saved checkpoint to confirm
    library match, then loads. Raises a clear message if mismatch.
    """
    ckpt = torch.load(path, map_location=DEVICE)

    # Handle if saved as {"model_state_dict": ...} or {"state_dict": ...}
    if isinstance(ckpt, dict):
        if "model_state_dict" in ckpt:
            ckpt = ckpt["model_state_dict"]
        elif "state_dict" in ckpt:
            ckpt = ckpt["state_dict"]
        # else assume it IS the state_dict already

    saved_keys   = list(ckpt.keys())[:3]
    model_keys   = list(model.state_dict().keys())[:3]
    print(f"\n{name}")
    print(f"  Saved keys (first 3) : {saved_keys}")
    print(f"  Model keys (first 3) : {model_keys}")

    result = model.load_state_dict(ckpt, strict=True)
    print(f"  ✅ Loaded successfully")
    return model


print("Loading trained models...\n")

model_b0 = EfficientNetB0().to(DEVICE)
model_b0 = inspect_and_load(model_b0, PATH_B0, "EfficientNet-B0")
model_b0.eval()
count_params(model_b0, "EfficientNet-B0")

model_b3 = EfficientNetB3().to(DEVICE)
model_b3 = inspect_and_load(model_b3, PATH_B3, "EfficientNet-B3")
model_b3.eval()
count_params(model_b3, "EfficientNet-B3")

model_vit = ViTBase().to(DEVICE)
model_vit = inspect_and_load(model_vit, PATH_VIT, "ViT-Base/16")
model_vit.eval()
count_params(model_vit, "ViT-Base/16")

print("\n✅ All three models loaded.")


Loading trained models...

Loaded pretrained weights for efficientnet-b0

EfficientNet-B0
  Saved keys (first 3) : ['net._conv_stem.weight', 'net._bn0.weight', 'net._bn0.bias']
  Model keys (first 3) : ['net._conv_stem.weight', 'net._bn0.weight', 'net._bn0.bias']
  ✅ Loaded successfully
EfficientNet-B0           4.0M total | 4.0M trainable
Loaded pretrained weights for efficientnet-b3

EfficientNet-B3
  Saved keys (first 3) : ['net._conv_stem.weight', 'net._bn0.weight', 'net._bn0.bias']
  Model keys (first 3) : ['net._conv_stem.weight', 'net._bn0.weight', 'net._bn0.bias']
  ✅ Loaded successfully
EfficientNet-B3           11.5M total | 11.5M trainable

ViT-Base/16
  Saved keys (first 3) : ['vit.cls_token', 'vit.pos_embed', 'vit.patch_embed.proj.weight']
  Model keys (first 3) : ['vit.cls_token', 'vit.pos_embed', 'vit.patch_embed.proj.weight']
  ✅ Loaded successfully
ViT-Base/16               86.2M total | 86.2M trainable

✅ All three models loaded.


In [39]:
# CELL 9 (FIXED): Handle NaN per-class AUC from single-class columns

def safe_roc_auc(y_true, y_pred, average=None):
    """
    Computes ROC AUC safely — skips classes with only one label present.
    Returns per-class array with np.nan for invalid classes,
    and macro average computed only over valid classes.
    """
    n_classes  = y_true.shape[1]
    per_class  = np.full(n_classes, np.nan)

    for i in range(n_classes):
        unique = np.unique(y_true[:, i])
        if len(unique) < 2:
            print(f"  ⚠️  Skipping {LABELS[i]} — only class {unique} present in split")
            continue
        per_class[i] = roc_auc_score(y_true[:, i], y_pred[:, i])

    valid_mask  = ~np.isnan(per_class)
    macro_auc   = float(np.mean(per_class[valid_mask]))
    return macro_auc, per_class


@torch.no_grad()
def evaluate_model(model, loader, label="Model"):
    model.eval()
    all_labels, all_preds = [], []
    for x, y, meta in tqdm(loader, desc=f"  Eval {label}", ncols=100):
        x = x.to(DEVICE)
        with torch.cuda.amp.autocast():
            logits = model(x, None)
        all_labels.append(y.numpy())
        all_preds.append(torch.sigmoid(logits).cpu().float().numpy())

    y_true = np.vstack(all_labels)
    y_pred = np.vstack(all_preds)

    auc_macro, per_class = safe_roc_auc(y_true, y_pred)

    print(f"\n{'='*55}")
    print(f"  {label}  —  Test AUC (macro): {auc_macro:.4f}")
    print(f"{'='*55}")
    for l, a in zip(LABELS, per_class):
        if np.isnan(a):
            print(f"  {l:<22}  N/A  (single class)")
        else:
            flag = "  ⚠️ " if a < 0.70 else ("  ✅" if a >= 0.85 else "")
            print(f"  {l:<22} {a:.4f}{flag}")

    return y_pred, y_true, auc_macro, per_class


# Also fix evaluate_model_with_meta in Cell 11
@torch.no_grad()
def evaluate_model_with_meta(model, loader, label="Model"):
    model.eval()
    all_labels, all_preds = [], []
    for x, y, meta in tqdm(loader, desc=f"  Eval {label}", ncols=100):
        x, meta = x.to(DEVICE), meta.to(DEVICE)
        with torch.cuda.amp.autocast():
            logits = model(x, meta)
        all_labels.append(y.numpy())
        all_preds.append(torch.sigmoid(logits).cpu().float().numpy())

    y_true = np.vstack(all_labels)
    y_pred = np.vstack(all_preds)

    auc_macro, per_class = safe_roc_auc(y_true, y_pred)

    print(f"\n{'='*55}")
    print(f"  {label}  —  Test AUC (macro): {auc_macro:.4f}")
    print(f"{'='*55}")
    for l, a in zip(LABELS, per_class):
        if np.isnan(a):
            print(f"  {l:<22}  N/A  (single class)")
        else:
            flag = "  ⚠️ " if a < 0.70 else ("  ✅" if a >= 0.85 else "")
            print(f"  {l:<22} {a:.4f}{flag}")

    return y_pred, y_true, auc_macro, per_class


# Also fix val_epoch in Cell 11
def val_epoch(model, loader, loss_fn):
    model.eval()
    total_loss, all_labels, all_preds = 0.0, [], []
    with torch.no_grad():
        for x, y, meta in tqdm(loader, desc="  Validate", ncols=110):
            x, y, meta = x.to(DEVICE), y.to(DEVICE), meta.to(DEVICE)
            with torch.cuda.amp.autocast():
                logits = model(x, meta)
                loss   = loss_fn(logits, y)
            total_loss += loss.item()
            all_labels.append(y.cpu().numpy())
            all_preds.append(torch.sigmoid(logits).cpu().float().numpy())
    y_true = np.vstack(all_labels)
    y_pred = np.vstack(all_preds)
    auc, per_class = safe_roc_auc(y_true, y_pred)
    return total_loss / len(loader), auc, per_class


# Fix safe_roc_auc for ensemble cells (scalar input)
def safe_roc_auc_ensemble(y_true, y_pred):
    """Same as safe_roc_auc but also handles the 3/4-way ensemble arrays."""
    n_classes = y_true.shape[1]
    per_class = np.full(n_classes, np.nan)
    for i in range(n_classes):
        if len(np.unique(y_true[:, i])) < 2:
            continue
        per_class[i] = roc_auc_score(y_true[:, i], y_pred[:, i])
    valid     = ~np.isnan(per_class)
    macro_auc = float(np.mean(per_class[valid]))
    return macro_auc, per_class

print("✅ Safe AUC functions defined. Re-running evaluations...\n")

# Re-run all 3 evaluations
print("Evaluating EfficientNet-B0...")
b0_preds,  test_labels, b0_auc,  b0_pc  = evaluate_model(model_b0,  TEST_DL, "EfficientNet-B0")

print("\nEvaluating EfficientNet-B3...")
b3_preds,  _,           b3_auc,  b3_pc  = evaluate_model(model_b3,  TEST_DL, "EfficientNet-B3")

print("\nEvaluating ViT-Base/16...")
vit_preds, _,           vit_auc, vit_pc = evaluate_model(model_vit, TEST_DL, "ViT-Base/16")

print(f"\n{'='*55}")
print(f"  EfficientNet-B0  AUC: {b0_auc:.4f}")
print(f"  EfficientNet-B3  AUC: {b3_auc:.4f}")
print(f"  ViT-Base/16      AUC: {vit_auc:.4f}")


✅ Safe AUC functions defined. Re-running evaluations...

Evaluating EfficientNet-B0...


  Eval EfficientNet-B0: 100%|███████████████████████████████████| 2560/2560 [28:06<00:00,  1.52it/s]


  ⚠️  Skipping PleuralThickening — only class [0.] present in split

  EfficientNet-B0  —  Test AUC (macro): 0.8293
  Atelectasis            0.7106
  Consolidation          0.7463
  Infiltration           0.7004
  Pneumothorax           0.8752  ✅
  Edema                  0.8988  ✅
  Emphysema              0.9347  ✅
  Fibrosis               0.9107  ✅
  Effusion               0.7849
  Pneumonia              0.8687  ✅
  PleuralThickening       N/A  (single class)
  Cardiomegaly           0.9142  ✅
  Nodule                 0.6995  ⚠️ 
  Mass                   0.7583
  Hernia                 0.9783  ✅

Evaluating EfficientNet-B3...


  Eval EfficientNet-B3: 100%|███████████████████████████████████| 2560/2560 [27:12<00:00,  1.57it/s]


  ⚠️  Skipping PleuralThickening — only class [0.] present in split

  EfficientNet-B3  —  Test AUC (macro): 0.8350
  Atelectasis            0.7096
  Consolidation          0.7448
  Infiltration           0.6867  ⚠️ 
  Pneumothorax           0.8780  ✅
  Edema                  0.9207  ✅
  Emphysema              0.9474  ✅
  Fibrosis               0.9365  ✅
  Effusion               0.7766
  Pneumonia              0.8834  ✅
  PleuralThickening       N/A  (single class)
  Cardiomegaly           0.9192  ✅
  Nodule                 0.7100
  Mass                   0.7579
  Hernia                 0.9843  ✅

Evaluating ViT-Base/16...


  Eval ViT-Base/16: 100%|███████████████████████████████████████| 2560/2560 [25:30<00:00,  1.67it/s]

  ⚠️  Skipping PleuralThickening — only class [0.] present in split

  ViT-Base/16  —  Test AUC (macro): 0.8379
  Atelectasis            0.7324
  Consolidation          0.7590
  Infiltration           0.6749  ⚠️ 
  Pneumothorax           0.8852  ✅
  Edema                  0.9067  ✅
  Emphysema              0.9386  ✅
  Fibrosis               0.9113  ✅
  Effusion               0.7945
  Pneumonia              0.8666  ✅
  PleuralThickening       N/A  (single class)
  Cardiomegaly           0.9239  ✅
  Nodule                 0.7245
  Mass                   0.7988
  Hernia                 0.9757  ✅

  EfficientNet-B0  AUC: 0.8293
  EfficientNet-B3  AUC: 0.8350
  ViT-Base/16      AUC: 0.8379


In [19]:
# CELL 10 (catch-up): 3-Way Ensemble — run before Cell 15

weight_configs_3 = [
    (0.40, 0.30, 0.30),
    (0.50, 0.25, 0.25),
    (0.40, 0.20, 0.40),
    (0.33, 0.33, 0.34),
    (0.35, 0.25, 0.40),
    (0.45, 0.15, 0.40),
]

best3_auc, best3_w, best3_preds = -1.0, None, None
print("Testing 3-way ensemble weights...\n")

for wb0, wb3, wvit in weight_configs_3:
    ens      = wb0*b0_preds + wb3*b3_preds + wvit*vit_preds
    auc, _   = safe_roc_auc(test_labels, ens)
    auc_safe = auc if not np.isnan(auc) else 0.0
    flag     = "  ← BEST" if auc_safe > best3_auc else ""
    print(f"  B0={wb0:.2f}  B3={wb3:.2f}  ViT={wvit:.2f}  →  AUC={auc_safe:.4f}{flag}")
    if auc_safe > best3_auc:
        best3_auc, best3_w, best3_preds = auc_safe, (wb0, wb3, wvit), ens.copy()

_, best3_pc  = safe_roc_auc(test_labels, best3_preds)
m_aucs_3     = [best3_pc[LABELS.index(c)] for c in minority_classes]

print(f"\n{'='*55}")
print(f"  Best 3-Way Ensemble AUC : {best3_auc:.4f}")
print(f"  Weights → B0={best3_w[0]}  B3={best3_w[1]}  ViT={best3_w[2]}")
print(f"{'='*55}")
print("\nPer-class AUC (3-Way Ensemble):")
for l, a in zip(LABELS, best3_pc):
    if np.isnan(a):
        print(f"  {l:<22}  N/A")
    else:
        flag = "  ⚠️ " if a < 0.70 else ("  ✅" if a >= 0.85 else "")
        print(f"  {l:<22} {a:.4f}{flag}")

print(f"\n  Minority Avg AUC : {np.nanmean(m_aucs_3):.4f}")
for c, a in zip(minority_classes, m_aucs_3):
    print(f"  {c:<22} {a:.4f}" if not np.isnan(a) else f"  {c:<22}  N/A")

print("\n✅ 3-Way ensemble done — now re-run Cell 15.")


Testing 3-way ensemble weights...

  B0=0.40  B3=0.30  ViT=0.30  →  AUC=0.8566  ← BEST
  B0=0.50  B3=0.25  ViT=0.25  →  AUC=0.8546
  B0=0.40  B3=0.20  ViT=0.40  →  AUC=0.8571  ← BEST
  B0=0.33  B3=0.33  ViT=0.34  →  AUC=0.8575  ← BEST
  B0=0.35  B3=0.25  ViT=0.40  →  AUC=0.8577  ← BEST
  B0=0.45  B3=0.15  ViT=0.40  →  AUC=0.8561

  Best 3-Way Ensemble AUC : 0.8577
  Weights → B0=0.35  B3=0.25  ViT=0.4

Per-class AUC (3-Way Ensemble):
  Atelectasis            0.7427
  Consolidation          0.7735
  Infiltration           0.7080
  Pneumothorax           0.8993  ✅
  Edema                  0.9343  ✅
  Emphysema              0.9599  ✅
  Fibrosis               0.9463  ✅
  Effusion               0.8111
  Pneumonia              0.9021  ✅
  PleuralThickening       N/A
  Cardiomegaly           0.9445  ✅
  Nodule                 0.7382
  Mass                   0.8074
  Hernia                 0.9831  ✅

  Minority Avg AUC : 0.9451
  Hernia                 0.9831
  Pneumonia              0.9021
  

In [10]:
# CELL 11 FIXED: All helpers with safe AUC — run before Cell 12

def safe_roc_auc(y_true, y_pred):
    """Skips classes with only one label, returns valid macro AUC."""
    n_classes = y_true.shape[1]
    per_class = np.full(n_classes, np.nan)
    for i in range(n_classes):
        if len(np.unique(y_true[:, i])) < 2:
            continue
        per_class[i] = roc_auc_score(y_true[:, i], y_pred[:, i])
    valid     = ~np.isnan(per_class)
    macro_auc = float(np.nanmean(per_class[valid]))
    return macro_auc, per_class


def train_epoch(model, loader, loss_fn, optimizer, scaler, scheduler, epoch):
    model.train()
    total_loss, all_labels, all_preds = 0.0, [], []
    pbar = tqdm(loader, desc=f"Epoch {epoch:02d} Train", ncols=110)
    for i, (x, y, meta) in enumerate(pbar):
        x, y, meta = x.to(DEVICE), y.to(DEVICE), meta.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast():
            logits = model(x, meta)
            loss   = loss_fn(logits, y)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        if scheduler:
            scheduler.step()
        total_loss += loss.item()
        all_labels.append(y.detach().cpu().numpy())
        all_preds.append(torch.sigmoid(logits).detach().cpu().float().numpy())
        if i % 300 == 0:
            pbar.set_postfix({
                "loss": f"{total_loss/(i+1):.4f}",
                "lr":   f"{optimizer.param_groups[-1]['lr']:.2e}"
            })
    # ✅ safe AUC — skips PleuralThickening if single-class
    auc, _ = safe_roc_auc(np.vstack(all_labels), np.vstack(all_preds))
    return total_loss / len(loader), auc


def val_epoch(model, loader, loss_fn):
    model.eval()
    total_loss, all_labels, all_preds = 0.0, [], []
    with torch.no_grad():
        for x, y, meta in tqdm(loader, desc="  Validate", ncols=110):
            x, y, meta = x.to(DEVICE), y.to(DEVICE), meta.to(DEVICE)
            with torch.cuda.amp.autocast():
                logits = model(x, meta)
                loss   = loss_fn(logits, y)
            total_loss += loss.item()
            all_labels.append(y.cpu().numpy())
            all_preds.append(torch.sigmoid(logits).cpu().float().numpy())
    y_true = np.vstack(all_labels)
    y_pred = np.vstack(all_preds)
    # ✅ safe AUC
    auc, per_class = safe_roc_auc(y_true, y_pred)
    return total_loss / len(loader), auc, per_class


@torch.no_grad()
def evaluate_model_with_meta(model, loader, label="Model"):
    model.eval()
    all_labels, all_preds = [], []
    for x, y, meta in tqdm(loader, desc=f"  Eval {label}", ncols=100):
        x, meta = x.to(DEVICE), meta.to(DEVICE)
        with torch.cuda.amp.autocast():
            logits = model(x, meta)
        all_labels.append(y.numpy())
        all_preds.append(torch.sigmoid(logits).cpu().float().numpy())
    y_true = np.vstack(all_labels)
    y_pred = np.vstack(all_preds)
    auc_macro, per_class = safe_roc_auc(y_true, y_pred)
    print(f"\n{'='*55}")
    print(f"  {label}  —  Test AUC (macro): {auc_macro:.4f}")
    print(f"{'='*55}")
    for l, a in zip(LABELS, per_class):
        if np.isnan(a):
            print(f"  {l:<22}  N/A  (single class in split)")
        else:
            flag = "  ⚠️ " if a < 0.70 else ("  ✅" if a >= 0.85 else "")
            print(f"  {l:<22} {a:.4f}{flag}")
    return y_pred, y_true, auc_macro, per_class

print("✅ All helpers defined correctly with safe AUC.")


✅ All helpers defined correctly with safe AUC.


In [45]:
# CELL 12 FIXED: Rebuild DINOv2 and retrain with correct AUC + saving logic

# Rebuild model fresh (previous run never saved a valid checkpoint)
print("Rebuilding DINOv2 Multi-Modal model...")
model_dino = DINOv2MultiModal(freeze_blocks=8).to(DEVICE)
count_params(model_dino, "DINOv2 MultiModal")

loss_fn = FocalLossWithSmoothing(CLASS_WEIGHTS, gamma=2.0, smoothing=LABEL_SMOOTH)

# Layer-wise LR
backbone_params, head_params = [], []
for name, param in model_dino.named_parameters():
    if not param.requires_grad:
        continue
    if any(k in name for k in ["backbone", "blocks", "norm", "patch_embed"]):
        backbone_params.append(param)
    else:
        head_params.append(param)

optimizer_dino = torch.optim.AdamW([
    {"params": backbone_params, "lr": 5e-6},
    {"params": head_params,     "lr": 1e-4},
], weight_decay=1e-4)

NUM_EPOCHS_DINO = 15
PATIENCE_DINO   = 5
total_steps     = NUM_EPOCHS_DINO * len(TRAIN_DL)

scheduler_dino = OneCycleLR(
    optimizer_dino,
    max_lr           = [5e-6, 1e-4],
    total_steps      = total_steps,
    pct_start        = 0.1,
    anneal_strategy  = "cos",
    div_factor       = 25.0,
    final_div_factor = 1000.0,
)

scaler_dino  = torch.cuda.amp.GradScaler()
best_auc_d   = -1.0     # ✅ start at -1 so first valid epoch always saves
patience_ctr = 0
dino_history = []

print(f"\nStarting DINOv2 training — max {NUM_EPOCHS_DINO} epochs\n")

for epoch in range(1, NUM_EPOCHS_DINO + 1):
    tr_loss, tr_auc           = train_epoch(model_dino, TRAIN_DL, loss_fn,
                                             optimizer_dino, scaler_dino,
                                             scheduler_dino, epoch)
    val_loss, val_auc, val_pc = val_epoch(model_dino, VAL_DL, loss_fn)

    # ✅ Guard against nan before comparison
    val_auc_safe = val_auc if not np.isnan(val_auc) else 0.0
    tr_auc_safe  = tr_auc  if not np.isnan(tr_auc)  else 0.0

    print(f"\nEpoch {epoch}/{NUM_EPOCHS_DINO}  |  "
          f"Train Loss {tr_loss:.4f}  AUC {tr_auc_safe:.4f}  |  "
          f"Val Loss {val_loss:.4f}  AUC {val_auc_safe:.4f}")

    for l, a in zip(LABELS, val_pc):
        if np.isnan(a):
            print(f"  {l:<22}  N/A")
        else:
            flag = "  ⚠️ " if a < 0.70 else ("  ✅" if a >= 0.85 else "")
            print(f"  {l:<22} {a:.4f}{flag}")

    dino_history.append({
        "epoch":         epoch,
        "train_loss":    tr_loss,
        "train_auc":     tr_auc_safe,
        "val_loss":      val_loss,
        "val_auc":       val_auc_safe,
        "per_class_auc": [float(a) if not np.isnan(a) else None
                          for a in val_pc],
    })

    # ✅ Safe comparison: only update if val_auc is a real number
    if not np.isnan(val_auc) and val_auc_safe > best_auc_d:
        best_auc_d   = val_auc_safe
        patience_ctr = 0
        torch.save(model_dino.state_dict(), PATH_DINO)
        print(f"  ✅ Best saved → {PATH_DINO}  (Val AUC {best_auc_d:.4f})")
    else:
        patience_ctr += 1
        print(f"  No improvement ({patience_ctr}/{PATIENCE_DINO})  "
              f"[best so far: {best_auc_d:.4f}]")
        if patience_ctr >= PATIENCE_DINO:
            print("  🛑 Early stopping triggered.")
            break

with open(os.path.join(BASE_DIR, "phase7_dinov2_results.json"), "w") as f:
    json.dump(dino_history, f, indent=2)

print(f"\n✅ DINOv2 training complete.  Best Val AUC: {best_auc_d:.4f}")

Rebuilding DINOv2 Multi-Modal model...


Using cache found in C:\Users\91850/.cache\torch\hub\facebookresearch_dinov2_main


DINOv2 MultiModal         87.6M total | 29.4M trainable

Starting DINOv2 training — max 15 epochs



  Validate: 100%|█████████████████████████████████████████████████████████| 1298/1298 [11:50<00:00,  1.83it/s]



Epoch 1/15  |  Train Loss 0.0075  AUC 0.5762  |  Val Loss 0.0023  AUC 0.7262
  Atelectasis            0.7051
  Consolidation          0.7484
  Infiltration           0.6039  ⚠️ 
  Pneumothorax           0.7712
  Edema                  0.8693  ✅
  Emphysema              0.8092
  Fibrosis               0.6924  ⚠️ 
  Effusion               0.7489
  Pneumonia              0.6281  ⚠️ 
  PleuralThickening       N/A
  Cardiomegaly           0.7893
  Nodule                 0.5702  ⚠️ 
  Mass                   0.6009  ⚠️ 
  Hernia                 0.9037  ✅
  ✅ Best saved → D:\Major Project\best_ultimate_dinov2.pth  (Val AUC 0.7262)


...


Epoch 8/15  |  Train Loss 0.0018  AUC 0.8826  |  Val Loss 0.0021  AUC 0.7703
  Atelectasis            0.7620
  Consolidation          0.7590
  Infiltration           0.6368  ⚠️ 
  Pneumothorax           0.8452
  Edema                  0.8292
  Emphysema              0.8942  ✅
  Fibrosis               0.7317
  Effusion               0.8611  ✅
  Pneumonia              0.6671  ⚠️ 
  PleuralThickening       N/A
  Cardiomegaly           0.8471
  Nodule                 0.6486  ⚠️ 
  Mass                   0.7850
  Hernia                 0.7469
  No improvement (3/5)  [best so far: 0.7835]


  Validate: 100%|█████████████████████████████████████████████████████████| 1298/1298 [13:15<00:00,  1.63it/s]



Epoch 9/15  |  Train Loss 0.0016  AUC 0.8905  |  Val Loss 0.0023  AUC 0.7721
  Atelectasis            0.7712
  Consolidation          0.7717
  Infiltration           0.6248  ⚠️ 
  Pneumothorax           0.8505  ✅
  Edema                  0.8151
  Emphysema              0.9007  ✅
  Fibrosis               0.7472
  Effusion               0.8661  ✅
  Pneumonia              0.6321  ⚠️ 
  PleuralThickening       N/A
  Cardiomegaly           0.8491
  Nodule                 0.6523  ⚠️ 
  Mass                   0.7729
  Hernia                 0.7828
  No improvement (4/5)  [best so far: 0.7835]


  Validate: 100%|█████████████████████████████████████████████████████████| 1298/1298 [13:25<00:00,  1.61it/s]



Epoch 10/15  |  Train Loss 0.0015  AUC 0.8972  |  Val Loss 0.0024  AUC 0.7717
  Atelectasis            0.7692
  Consolidation          0.7604
  Infiltration           0.6247  ⚠️ 
  Pneumothorax           0.8493
  Edema                  0.8375
  Emphysema              0.8988  ✅
  Fibrosis               0.7341
  Effusion               0.8626  ✅
  Pneumonia              0.6284  ⚠️ 
  PleuralThickening       N/A
  Cardiomegaly           0.8523  ✅
  Nodule                 0.6535  ⚠️ 
  Mass                   0.7681
  Hernia                 0.7938
  No improvement (5/5)  [best so far: 0.7835]
  🛑 Early stopping triggered.

✅ DINOv2 training complete.  Best Val AUC: 0.7835


In [46]:
# CELL 13: Load best DINOv2 checkpoint and evaluate on test set

print(f"Loading best DINOv2 weights from:\n  {PATH_DINO}\n")
model_dino.load_state_dict(torch.load(PATH_DINO, map_location=DEVICE))

dino_preds, _, dino_auc, dino_pc = evaluate_model_with_meta(
    model_dino, TEST_DL, "DINOv2 MultiModal"
)

m_aucs_dino = [dino_pc[LABELS.index(c)] for c in minority_classes]
print(f"\n  Minority Avg AUC : {np.mean(m_aucs_dino):.4f}")
for c, a in zip(minority_classes, m_aucs_dino):
    print(f"  {c:<22} {a:.4f}")

Loading best DINOv2 weights from:
  D:\Major Project\best_ultimate_dinov2.pth



  Eval DINOv2 MultiModal: 100%|█████████████████████████████████| 2560/2560 [26:53<00:00,  1.59it/s]


  DINOv2 MultiModal  —  Test AUC (macro): 0.7624
  Atelectasis            0.7141
  Consolidation          0.7171
  Infiltration           0.6746  ⚠️ 
  Pneumothorax           0.8505  ✅
  Edema                  0.8090
  Emphysema              0.9107  ✅
  Fibrosis               0.7476
  Effusion               0.7752
  Pneumonia              0.6160  ⚠️ 
  PleuralThickening       N/A  (single class in split)
  Cardiomegaly           0.8372
  Nodule                 0.6609  ⚠️ 
  Mass                   0.7291
  Hernia                 0.8697  ✅

  Minority Avg AUC : 0.7906
  Hernia                 0.8697
  Pneumonia              0.6160
  Edema                  0.8090
  Emphysema              0.9107
  Fibrosis               0.7476


In [14]:
# Load DINOv2 best checkpoint

model_dino = DINOv2MultiModal(freeze_blocks=8).to(DEVICE)
model_dino.load_state_dict(torch.load(PATH_DINO, map_location=DEVICE))
model_dino.eval()
count_params(model_dino, "DINOv2 MultiModal")

print("✅ DINOv2 loaded.")


Using cache found in C:\Users\91850/.cache\torch\hub\facebookresearch_dinov2_main


DINOv2 MultiModal         87.6M total | 29.4M trainable
✅ DINOv2 loaded.


In [15]:
# QUICK VERIFY CELL — run this first to see what's missing
checks = {
    "DEVICE / LABELS / BASE_DIR":     lambda: bool(DEVICE and LABELS and BASE_DIR),
    "TEST_DL":                         lambda: TEST_DL is not None,
    "model_b0":                        lambda: model_b0 is not None,
    "model_b3":                        lambda: model_b3 is not None,
    "model_vit":                       lambda: model_vit is not None,
    "model_dino":                      lambda: model_dino is not None,
    "safe_roc_auc":                    lambda: callable(safe_roc_auc),
}

all_ok = True
for name, check in checks.items():
    try:
        ok = check()
        print(f"  {'✅' if ok else '❌'} {name}")
        if not ok:
            all_ok = False
    except NameError:
        print(f"  ❌ {name}  ← NOT DEFINED")
        all_ok = False

print(f"\n{'✅ All ready — run Resume Cell' if all_ok else '❌ Fix missing items above first'}")


  ✅ DEVICE / LABELS / BASE_DIR
  ✅ TEST_DL
  ✅ model_b0
  ✅ model_b3
  ✅ model_vit
  ✅ model_dino
  ✅ safe_roc_auc

✅ All ready — run Resume Cell


In [16]:
# RESUME CELL: Regenerate all predictions after reopening notebook
# Run AFTER cells 1,2,3,4,5,7,8,11 have been re-executed

import os
import numpy as np

RESUME_CACHE_DIR = BASE_DIR   # predictions saved here as .npy files
print("Checking for cached predictions...\n")

# ── Helper: try to load from cache, else recompute ──────────────────────────
def load_or_compute(cache_path, compute_fn, label):
    if os.path.exists(cache_path):
        arr = np.load(cache_path)
        print(f"  ✅ Loaded from cache : {os.path.basename(cache_path)}")
        return arr
    else:
        print(f"  🔄 Computing         : {label}")
        arr = compute_fn()
        np.save(cache_path, arr)
        print(f"  💾 Saved to cache    : {os.path.basename(cache_path)}")
        return arr


# ── Predict functions ────────────────────────────────────────────────────────
@torch.no_grad()
def get_test_preds_no_meta(model):
    model.eval()
    preds = []
    for x, y, meta in tqdm(TEST_DL, ncols=100):
        x = x.to(DEVICE)
        with torch.cuda.amp.autocast():
            logits = model(x, None)
        preds.append(torch.sigmoid(logits).cpu().float().numpy())
    return np.vstack(preds)


@torch.no_grad()
def get_test_preds_with_meta(model):
    model.eval()
    preds = []
    for x, y, meta in tqdm(TEST_DL, ncols=100):
        x, meta = x.to(DEVICE), meta.to(DEVICE)
        with torch.cuda.amp.autocast():
            logits = model(x, meta)
        preds.append(torch.sigmoid(logits).cpu().float().numpy())
    return np.vstack(preds)


@torch.no_grad()
def get_test_labels():
    labels = []
    for x, y, meta in tqdm(TEST_DL, desc="Labels", ncols=100):
        labels.append(y.numpy())
    return np.vstack(labels)


# ── Load or compute all predictions ─────────────────────────────────────────
test_labels = load_or_compute(
    os.path.join(RESUME_CACHE_DIR, "cache_test_labels.npy"),
    get_test_labels, "test labels"
)

b0_preds = load_or_compute(
    os.path.join(RESUME_CACHE_DIR, "cache_b0_preds.npy"),
    lambda: get_test_preds_no_meta(model_b0), "EfficientNet-B0"
)

b3_preds = load_or_compute(
    os.path.join(RESUME_CACHE_DIR, "cache_b3_preds.npy"),
    lambda: get_test_preds_no_meta(model_b3), "EfficientNet-B3"
)

vit_preds = load_or_compute(
    os.path.join(RESUME_CACHE_DIR, "cache_vit_preds.npy"),
    lambda: get_test_preds_no_meta(model_vit), "ViT-Base/16"
)

dino_preds = load_or_compute(
    os.path.join(RESUME_CACHE_DIR, "cache_dino_preds.npy"),
    lambda: get_test_preds_with_meta(model_dino), "DINOv2"
)


# ── Recompute per-model AUCs ─────────────────────────────────────────────────
b0_auc,   b0_pc   = safe_roc_auc(test_labels, b0_preds)
b3_auc,   b3_pc   = safe_roc_auc(test_labels, b3_preds)
vit_auc,  vit_pc  = safe_roc_auc(test_labels, vit_preds)
dino_auc, dino_pc = safe_roc_auc(test_labels, dino_preds)

minority_classes = ["Hernia","Pneumonia","Edema","Emphysema","Fibrosis"]

print(f"\n{'='*55}")
print(f"  {'Model':<28} {'AUC':>8}")
print(f"  {'-'*40}")
print(f"  {'EfficientNet-B0':<28} {b0_auc:>8.4f}")
print(f"  {'EfficientNet-B3':<28} {b3_auc:>8.4f}")
print(f"  {'ViT-Base/16':<28} {vit_auc:>8.4f}")
print(f"  {'DINOv2 MultiModal':<28} {dino_auc:>8.4f}")
print(f"{'='*55}")
print("\n✅ All predictions ready — continue from Cell 14.")


Checking for cached predictions...

  ✅ Loaded from cache : cache_test_labels.npy
  🔄 Computing         : EfficientNet-B0


100%|███████████████████████████████████████████████████████████| 2560/2560 [28:59<00:00,  1.47it/s]


  💾 Saved to cache    : cache_b0_preds.npy
  🔄 Computing         : EfficientNet-B3


100%|███████████████████████████████████████████████████████████| 2560/2560 [31:10<00:00,  1.37it/s]


  💾 Saved to cache    : cache_b3_preds.npy
  🔄 Computing         : ViT-Base/16


100%|███████████████████████████████████████████████████████████| 2560/2560 [30:48<00:00,  1.38it/s]


  💾 Saved to cache    : cache_vit_preds.npy
  🔄 Computing         : DINOv2


100%|███████████████████████████████████████████████████████████| 2560/2560 [33:07<00:00,  1.29it/s]


  💾 Saved to cache    : cache_dino_preds.npy

  Model                             AUC
  ----------------------------------------
  EfficientNet-B0                0.8293
  EfficientNet-B3                0.8350
  ViT-Base/16                    0.8379
  DINOv2 MultiModal              0.7624

✅ All predictions ready — continue from Cell 14.


In [17]:
# CELL 14 FIXED: 4-Way Ensemble using safe_roc_auc (already defined in Cell 11)

weight_configs_4 = [
    (0.25, 0.15, 0.25, 0.35),
    (0.30, 0.10, 0.25, 0.35),
    (0.20, 0.15, 0.30, 0.35),
    (0.25, 0.15, 0.30, 0.30),
    (0.25, 0.20, 0.20, 0.35),
    (0.20, 0.10, 0.30, 0.40),
    (0.30, 0.15, 0.20, 0.35),
    (0.20, 0.10, 0.25, 0.45),
]

best4_auc, best4_w, best4_preds = -1.0, None, None
print("Testing 4-way ensemble weights...\n")

for wb0, wb3, wvit, wdino in weight_configs_4:
    ens       = wb0*b0_preds + wb3*b3_preds + wvit*vit_preds + wdino*dino_preds
    auc, _    = safe_roc_auc(test_labels, ens)           # ✅ use safe_roc_auc
    auc_safe  = auc if not np.isnan(auc) else 0.0        # ✅ guard nan
    flag      = "  ← BEST" if auc_safe > best4_auc else ""
    print(f"  B0={wb0:.2f}  B3={wb3:.2f}  ViT={wvit:.2f}  "
          f"DINO={wdino:.2f}  →  AUC={auc_safe:.4f}{flag}")
    if auc_safe > best4_auc:
        best4_auc   = auc_safe
        best4_w     = (wb0, wb3, wvit, wdino)
        best4_preds = ens.copy()                         # ✅ always set together

# Final safety check
if best4_preds is None:
    print("\n⚠️  All configs returned nan — using equal weights as fallback")
    best4_w     = (0.25, 0.25, 0.25, 0.25)
    best4_preds = 0.25*b0_preds + 0.25*b3_preds + 0.25*vit_preds + 0.25*dino_preds
    best4_auc, _ = safe_roc_auc(test_labels, best4_preds)

_, best4_pc  = safe_roc_auc(test_labels, best4_preds)
m_aucs_4     = [best4_pc[LABELS.index(c)] for c in minority_classes]

print(f"\n{'='*55}")
print(f"  Best 4-Way Ensemble AUC : {best4_auc:.4f}")
print(f"  Weights → B0={best4_w[0]}  B3={best4_w[1]}  "
      f"ViT={best4_w[2]}  DINO={best4_w[3]}")
print(f"{'='*55}")

print("\nPer-class AUC (4-Way Ensemble):")
for l, a in zip(LABELS, best4_pc):
    if np.isnan(a):
        print(f"  {l:<22}  N/A")
    else:
        flag = "  ⚠️ " if a < 0.70 else ("  ✅" if a >= 0.85 else "")
        print(f"  {l:<22} {a:.4f}{flag}")

print(f"\n  Minority Avg AUC : {np.nanmean(m_aucs_4):.4f}")
for c, a in zip(minority_classes, m_aucs_4):
    val_str = f"{a:.4f}" if not np.isnan(a) else "N/A"
    print(f"  {c:<22} {val_str}")


Testing 4-way ensemble weights...

  B0=0.25  B3=0.15  ViT=0.25  DINO=0.35  →  AUC=0.8576  ← BEST
  B0=0.30  B3=0.10  ViT=0.25  DINO=0.35  →  AUC=0.8566
  B0=0.20  B3=0.15  ViT=0.30  DINO=0.35  →  AUC=0.8580  ← BEST
  B0=0.25  B3=0.15  ViT=0.30  DINO=0.30  →  AUC=0.8592  ← BEST
  B0=0.25  B3=0.20  ViT=0.20  DINO=0.35  →  AUC=0.8574
  B0=0.20  B3=0.10  ViT=0.30  DINO=0.40  →  AUC=0.8554
  B0=0.30  B3=0.15  ViT=0.20  DINO=0.35  →  AUC=0.8567
  B0=0.20  B3=0.10  ViT=0.25  DINO=0.45  →  AUC=0.8528

  Best 4-Way Ensemble AUC : 0.8592
  Weights → B0=0.25  B3=0.15  ViT=0.3  DINO=0.3

Per-class AUC (4-Way Ensemble):
  Atelectasis            0.7533
  Consolidation          0.7703
  Infiltration           0.7189
  Pneumothorax           0.9002  ✅
  Edema                  0.9231  ✅
  Emphysema              0.9570  ✅
  Fibrosis               0.9424  ✅
  Effusion               0.8159
  Pneumonia              0.8963  ✅
  PleuralThickening       N/A
  Cardiomegaly           0.9323  ✅
  Nodule        

In [20]:
# CELL 15 FIXED: Summary table with nanmean for minority averages

print(f"\n{'='*68}")
print(f"  {'Model':<32} {'AUC':>8}  {'Minority Avg':>14}")
print(f"  {'-'*63}")
print(f"  {'EfficientNet-B0':<32} {b0_auc:>8.4f}")
print(f"  {'EfficientNet-B3':<32} {b3_auc:>8.4f}")
print(f"  {'ViT-Base/16':<32} {vit_auc:>8.4f}")
print(f"  {'DINOv2 MultiModal':<32} {dino_auc:>8.4f}")
print(f"  {'-'*63}")
print(f"  {'3-Way Ensemble (B0+B3+ViT)':<32} {best3_auc:>8.4f}"
      f"  {np.nanmean(m_aucs_3):>14.4f}")
print(f"  {'4-Way Ensemble (All)':<32} {best4_auc:>8.4f}"
      f"  {np.nanmean(m_aucs_4):>14.4f}")
print(f"{'='*68}")

# Per-class comparison
print(f"\n{'Per-Class AUC Comparison':^68}")
print(f"  {'Disease':<22} {'B0':>7} {'B3':>7} {'ViT':>7} "
      f"{'DINOv2':>7} {'3-Way':>7} {'4-Way':>7}")
print(f"  {'-'*63}")
for i, l in enumerate(LABELS):
    def fmt(v):
        return f"{v:>7.4f}" if not np.isnan(v) else "    N/A"
    print(f"  {l:<22}"
          f" {fmt(b0_pc[i])}"
          f" {fmt(b3_pc[i])}"
          f" {fmt(vit_pc[i])}"
          f" {fmt(dino_pc[i])}"
          f" {fmt(best3_pc[i])}"
          f" {fmt(best4_pc[i])}")

# Save results JSON
final_results = {
    "individual_models": {
        "EfficientNet-B0":   {
            "test_auc_macro": b0_auc,
            "per_class": {l: (float(a) if not np.isnan(a) else None)
                          for l, a in zip(LABELS, b0_pc)}
        },
        "EfficientNet-B3":   {
            "test_auc_macro": b3_auc,
            "per_class": {l: (float(a) if not np.isnan(a) else None)
                          for l, a in zip(LABELS, b3_pc)}
        },
        "ViT-Base16":        {
            "test_auc_macro": vit_auc,
            "per_class": {l: (float(a) if not np.isnan(a) else None)
                          for l, a in zip(LABELS, vit_pc)}
        },
        "DINOv2-MultiModal": {
            "test_auc_macro": dino_auc,
            "per_class": {l: (float(a) if not np.isnan(a) else None)
                          for l, a in zip(LABELS, dino_pc)}
        },
    },
    "ensemble_3way": {
        "weights":        dict(zip(["B0","B3","ViT"], best3_w)),
        "test_auc_macro": best3_auc,
        "per_class_auc":  {l: (float(a) if not np.isnan(a) else None)
                           for l, a in zip(LABELS, best3_pc)},
        "minority_avg":   float(np.nanmean(m_aucs_3)),
        "minority_detail":{ c: (float(a) if not np.isnan(a) else None)
                            for c, a in zip(minority_classes, m_aucs_3)},
    },
    "ensemble_4way": {
        "weights":        dict(zip(["B0","B3","ViT","DINOv2"], best4_w)),
        "test_auc_macro": best4_auc,
        "per_class_auc":  {l: (float(a) if not np.isnan(a) else None)
                           for l, a in zip(LABELS, best4_pc)},
        "minority_avg":   float(np.nanmean(m_aucs_4)),
        "minority_detail":{ c: (float(a) if not np.isnan(a) else None)
                            for c, a in zip(minority_classes, m_aucs_4)},
    },
    "labels": LABELS,
}

out_path = os.path.join(BASE_DIR, "FINAL_4WAY_ENSEMBLE_RESULTS.json")
with open(out_path, "w") as f:
    json.dump(final_results, f, indent=2)
print(f"\n✅ Results saved → {out_path}")



  Model                                 AUC    Minority Avg
  ---------------------------------------------------------------
  EfficientNet-B0                    0.8293
  EfficientNet-B3                    0.8350
  ViT-Base/16                        0.8379
  DINOv2 MultiModal                  0.7624
  ---------------------------------------------------------------
  3-Way Ensemble (B0+B3+ViT)         0.8577          0.9451
  4-Way Ensemble (All)               0.8592          0.9415

                      Per-Class AUC Comparison                      
  Disease                     B0      B3     ViT  DINOv2   3-Way   4-Way
  ---------------------------------------------------------------
  Atelectasis             0.7106  0.7096  0.7324  0.7141  0.7427  0.7533
  Consolidation           0.7463  0.7448  0.7590  0.7171  0.7735  0.7703
  Infiltration            0.7004  0.6867  0.6749  0.6746  0.7080  0.7189
  Pneumothorax            0.8752  0.8780  0.8852  0.8505  0.8993  0.9002
  Edema   

In [21]:
# CELL 16: Save per-image prediction CSVs for both ensembles

def save_predictions_csv(preds, dataset, filename):
    rows = []
    for i in range(len(preds)):
        row = {"Image_Index": dataset.df.iloc[i]["Image_Index"]}
        for j, lbl in enumerate(LABELS):
            row[lbl] = round(float(preds[i][j]), 4)
        top_idx = int(np.argmax(preds[i]))
        row["Predicted_Label"]      = LABELS[top_idx]
        row["Predicted_Confidence"] = round(float(preds[i][top_idx]), 4)
        rows.append(row)
    df_out = pd.DataFrame(rows)
    path   = os.path.join(BASE_DIR, filename)
    df_out.to_csv(path, index=False)
    print(f"  Saved {len(df_out):,} rows → {path}")
    return df_out

print("Saving prediction CSVs...\n")
test_dataset = TEST_DL.dataset

df_3way = save_predictions_csv(best3_preds, test_dataset, "predictions_3way_ensemble.csv")
df_4way = save_predictions_csv(best4_preds, test_dataset, "predictions_4way_ensemble.csv")

print("\nSample (4-Way Ensemble, first 10 rows):")
print(df_4way[["Image_Index","Predicted_Label","Predicted_Confidence"]]
      .head(10).to_string(index=False))


Saving prediction CSVs...

  Saved 25,596 rows → D:\Major Project\predictions_3way_ensemble.csv
  Saved 25,596 rows → D:\Major Project\predictions_4way_ensemble.csv

Sample (4-Way Ensemble, first 10 rows):
     Image_Index Predicted_Label  Predicted_Confidence
00000003_000.png          Hernia                0.8775
00000003_001.png        Fibrosis                0.3004
00000003_002.png          Hernia                0.7224
00000003_003.png          Hernia                0.7286
00000003_004.png          Hernia                0.7428
00000003_005.png          Hernia                0.9085
00000003_006.png          Hernia                0.8103
00000003_007.png          Hernia                0.7416
00000013_000.png    Cardiomegaly                0.3431
00000013_001.png       Emphysema                0.7298


In [22]:
# CELL 17: Per-class threshold optimisation via Youden's Index on validation set

from sklearn.metrics import roc_curve

print("Computing optimal thresholds on validation set...\n")

@torch.no_grad()
def get_preds(model, loader, use_meta=False):
    model.eval()
    all_labels, all_preds = [], []
    for x, y, meta in tqdm(loader, desc="  Preds", ncols=100):
        x, meta = x.to(DEVICE), meta.to(DEVICE)
        with torch.cuda.amp.autocast():
            logits = model(x, meta) if use_meta else model(x, None)
        all_labels.append(y.numpy())
        all_preds.append(torch.sigmoid(logits).cpu().float().numpy())
    return np.vstack(all_labels), np.vstack(all_preds)

# Validation predictions for all 4 models
print("Getting validation predictions...")
val_y, val_b0_p   = get_preds(model_b0,   VAL_DL, use_meta=False)
_,     val_b3_p   = get_preds(model_b3,   VAL_DL, use_meta=False)
_,     val_vit_p  = get_preds(model_vit,  VAL_DL, use_meta=False)
_,     val_dino_p = get_preds(model_dino, VAL_DL, use_meta=True)

wb0, wb3, wvit, wdino = best4_w
val_ens = wb0*val_b0_p + wb3*val_b3_p + wvit*val_vit_p + wdino*val_dino_p

# Youden's Index per class
optimal_thresholds = {}
print(f"\n  {'Disease':<22}  {'Threshold':>10}  {'Val AUC':>9}")
print(f"  {'-'*47}")
for i, lbl in enumerate(LABELS):
    fpr, tpr, thresholds = roc_curve(val_y[:, i], val_ens[:, i])
    best_idx              = int(np.argmax(tpr - fpr))
    best_thr              = float(thresholds[best_idx])
    auc_val               = roc_auc_score(val_y[:, i], val_ens[:, i])
    optimal_thresholds[lbl] = best_thr
    print(f"  {lbl:<22}  {best_thr:>10.4f}  {auc_val:>9.4f}")

thr_path = os.path.join(BASE_DIR, "optimized_thresholds.json")
with open(thr_path, "w") as f:
    json.dump(optimal_thresholds, f, indent=2)
print(f"\n✅ Thresholds saved → {thr_path}")


Computing optimal thresholds on validation set...

Getting validation predictions...


  Preds: 100%|██████████████████████████████████████████████████| 1298/1298 [10:22<00:00,  2.09it/s]



  Disease                  Threshold    Val AUC
  -----------------------------------------------
  Atelectasis                 0.2662     0.7980
  Consolidation               0.2190     0.8326
  Infiltration                0.3030     0.6668
  Pneumothorax                0.2048     0.8812
  Edema                       0.1973     0.9425
  Emphysema                   0.2064     0.9649
  Fibrosis                    0.1925     0.9260
  Effusion                    0.2973     0.8894
  Pneumonia                   0.2196     0.9283
  PleuralThickening              inf        nan
  Cardiomegaly                0.2015     0.9370
  Nodule                      0.2679     0.7451
  Mass                        0.2451     0.8441
  Hernia                      0.0153     0.9911

✅ Thresholds saved → D:\Major Project\optimized_thresholds.json


In [23]:
# CELL 18: Single-image inference with 4-way ensemble + optimized thresholds

def predict_single_image(image_path,
                         patient_age=50,
                         patient_gender="M",
                         view_position="PA",
                         followup=0):
    transform = build_transforms(train=False)
    img   = np.array(PILImage.open(image_path).convert("RGB"))
    img_t = transform(image=img)["image"].unsqueeze(0).to(DEVICE)

    age    = np.clip(patient_age, 0, 95) / 95.0
    gender = {"M": 0.0, "F": 1.0}.get(patient_gender.upper(), 0.5)
    view   = {"PA": 0.0, "AP": 1.0}.get(view_position.upper(), 0.5)
    fup    = np.clip(followup / 10.0, 0, 1)
    meta_t = torch.FloatTensor([[age, gender, view, fup]]).to(DEVICE)

    with torch.no_grad():
        with torch.cuda.amp.autocast():
            p_b0   = torch.sigmoid(model_b0(img_t,  None)).cpu().float().numpy()[0]
            p_b3   = torch.sigmoid(model_b3(img_t,  None)).cpu().float().numpy()[0]
            p_vit  = torch.sigmoid(model_vit(img_t, None)).cpu().float().numpy()[0]
            p_dino = torch.sigmoid(model_dino(img_t, meta_t)).cpu().float().numpy()[0]

    wb0, wb3, wvit, wdino = best4_w
    probs = wb0*p_b0 + wb3*p_b3 + wvit*p_vit + wdino*p_dino

    results = {}
    for i, lbl in enumerate(LABELS):
        prob = float(probs[i])
        thr  = optimal_thresholds.get(lbl, 0.5)
        results[lbl] = {
            "probability": round(prob, 4),
            "threshold":   round(thr, 4),
            "positive":    bool(prob >= thr),
        }

    results = dict(sorted(results.items(),
                          key=lambda x: x[1]["probability"], reverse=True))

    print(f"\nPredictions → {os.path.basename(image_path)}")
    print(f"{'='*54}")
    print(f"  {'Disease':<22}  {'Prob':>6}  {'Thr':>6}  {'Result'}")
    print(f"  {'-'*50}")
    for lbl, info in results.items():
        result_str = "POSITIVE ✅" if info["positive"] else "negative"
        print(f"  {lbl:<22}  {info['probability']:>6.4f}  "
              f"{info['threshold']:>6.4f}  {result_str}")

    positives = [l for l, v in results.items() if v["positive"]]
    print(f"\n  Detected: {positives if positives else ['No Finding']}")
    return results


# Test on a random image from the dataset
from PIL import Image as PILImage
sample_img = list(IMAGE_MAP.values())[42]
print(f"Testing inference on:\n  {sample_img}\n")
_ = predict_single_image(sample_img, patient_age=58,
                          patient_gender="M", view_position="PA")


Testing inference on:
  D:\Major Project\chest_xray_data\images_001\images\00000013_004.png


Predictions → 00000013_004.png
  Disease                   Prob     Thr  Result
  --------------------------------------------------
  Emphysema               0.6123  0.2064  POSITIVE ✅
  Pneumothorax            0.4619  0.2048  POSITIVE ✅
  Infiltration            0.3185  0.3030  POSITIVE ✅
  Effusion                0.3149  0.2973  POSITIVE ✅
  Atelectasis             0.3101  0.2662  POSITIVE ✅
  Nodule                  0.2429  0.2679  negative
  Mass                    0.2352  0.2451  negative
  Consolidation           0.2306  0.2190  POSITIVE ✅
  PleuralThickening       0.1944     inf  negative
  Cardiomegaly            0.1202  0.2015  negative
  Fibrosis                0.0867  0.1925  negative
  Pneumonia               0.0809  0.2196  negative
  Edema                   0.0570  0.1973  negative
  Hernia                  0.0113  0.0153  negative

  Detected: ['Emphysema', 'Pneumothorax', 'Inf

In [25]:
# CELL 19 FIXED: Load dino_history from saved JSON, then plot curves

import json
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ── Load history from disk (saved at end of Cell 12) ──────────────────────
history_path = os.path.join(BASE_DIR, "phase7_dinov2_results.json")

with open(history_path, "r") as f:
    dino_history = json.load(f)

print(f"✅ Loaded {len(dino_history)} epochs from {history_path}\n")
for h in dino_history:
    print(f"  Epoch {h['epoch']:>2}  |  "
          f"Train Loss {h['train_loss']:.4f}  AUC {h['train_auc']:.4f}  |  "
          f"Val Loss {h['val_loss']:.4f}  AUC {h['val_auc']:.4f}")

# ── Plot ───────────────────────────────────────────────────────────────────
epochs     = [h["epoch"]      for h in dino_history]
train_aucs = [h["train_auc"]  for h in dino_history]
val_aucs   = [h["val_auc"]    for h in dino_history]
train_loss = [h["train_loss"] for h in dino_history]
val_loss   = [h["val_loss"]   for h in dino_history]

best_val   = max(val_aucs)
best_epoch = epochs[val_aucs.index(best_val)]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# AUC plot
axes[0].plot(epochs, train_aucs, "b-o", label="Train AUC")
axes[0].plot(epochs, val_aucs,   "r-o", label="Val AUC")
axes[0].axvline(x=best_epoch, color="green", linestyle="--",
                label=f"Best epoch {best_epoch}")
axes[0].axhline(y=best3_auc,  color="purple", linestyle=":",
                label=f"3-Way Ens {best3_auc:.4f}")
axes[0].axhline(y=best4_auc,  color="orange", linestyle=":",
                label=f"4-Way Ens {best4_auc:.4f}")
axes[0].set_title("DINOv2 — AUC per Epoch")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("AUC (macro)")
axes[0].legend(fontsize=8)
axes[0].grid(True)
axes[0].set_xticks(epochs)

# Loss plot
axes[1].plot(epochs, train_loss, "b-o", label="Train Loss")
axes[1].plot(epochs, val_loss,   "r-o", label="Val Loss")
axes[1].axvline(x=best_epoch, color="green", linestyle="--",
                label=f"Best epoch {best_epoch}")
axes[1].set_title("DINOv2 — Focal Loss per Epoch")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Focal Loss")
axes[1].legend(fontsize=8)
axes[1].grid(True)
axes[1].set_xticks(epochs)

plt.suptitle(
    f"Phase 7 DINOv2 Training  |  "
    f"Best Val AUC: {best_val:.4f} (Epoch {best_epoch})  |  "
    f"4-Way Ensemble: {best4_auc:.4f}",
    fontsize=12
)
plt.tight_layout()

fig_path = os.path.join(BASE_DIR, "phase7_dinov2_training_curves.png")
plt.savefig(fig_path, dpi=150, bbox_inches="tight")
plt.close()
print(f"\n✅ Training curves saved → {fig_path}")


✅ Loaded 10 epochs from D:\Major Project\phase7_dinov2_results.json

  Epoch  1  |  Train Loss 0.0075  AUC 0.5762  |  Val Loss 0.0023  AUC 0.7262
  Epoch  2  |  Train Loss 0.0038  AUC 0.7213  |  Val Loss 0.0017  AUC 0.7701
  Epoch  3  |  Train Loss 0.0033  AUC 0.7746  |  Val Loss 0.0019  AUC 0.7741
  Epoch  4  |  Train Loss 0.0029  AUC 0.8118  |  Val Loss 0.0018  AUC 0.7776
  Epoch  5  |  Train Loss 0.0025  AUC 0.8389  |  Val Loss 0.0017  AUC 0.7835
  Epoch  6  |  Train Loss 0.0022  AUC 0.8592  |  Val Loss 0.0020  AUC 0.7763
  Epoch  7  |  Train Loss 0.0019  AUC 0.8717  |  Val Loss 0.0021  AUC 0.7762
  Epoch  8  |  Train Loss 0.0018  AUC 0.8826  |  Val Loss 0.0021  AUC 0.7703
  Epoch  9  |  Train Loss 0.0016  AUC 0.8905  |  Val Loss 0.0023  AUC 0.7721
  Epoch 10  |  Train Loss 0.0015  AUC 0.8972  |  Val Loss 0.0024  AUC 0.7717

✅ Training curves saved → D:\Major Project\phase7_dinov2_training_curves.png
